In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import substring, length, col, expr, when
from pyspark.sql.types import IntegerType
from pyspark.sql import Window

In [0]:
file_location_geo = "mnt/0e36c8cd403d_s3_bucket/topics/0e36c8cd403d.geo/partition=0/*.json" 
file_location_pin = "mnt/0e36c8cd403d_s3_bucket/topics/0e36c8cd403d.pin/partition=0/*.json" 
file_location_user = "mnt/0e36c8cd403d_s3_bucket/topics/0e36c8cd403d.user/partition=0/*.json" 

file_type = "json"
infer_schema = "true"

df_geo_pre_cleaning = (spark.read.format(file_type)
                       .option("inferSchema", infer_schema)
                       .load(file_location_geo)
)
df_pin_pre_cleaning = (spark.read.format(file_type)
                       .option("inferSchema", infer_schema)
                       .load(file_location_pin)
)
df_user_pre_cleaning = (spark.read.format(file_type)
                        .option("inferSchema", infer_schema)
                        .load(file_location_user)
)

In [0]:
df_pin_cleaned = (
    # Drop duplicates
    df_pin_pre_cleaning.drop_duplicates([column_name for column_name, data_type in df_pin_pre_cleaning.dtypes])
        # Replace entries with no relevant data in each column with Nones
        .withColumn("follower_count", F.when(F.col("follower_count") == "User Info Error", None)
                    .otherwise(F.col("follower_count")))
        .withColumn("image_src", F.when(F.col("image_src") == "Image src error.", None)
                    .otherwise(F.col("image_src")))
        .withColumn("poster_name", F.when(F.col("poster_name") == "User Info Error", None)
                    .otherwise(F.col("poster_name")))
        
        # Convert follower_count columns with k/M
        .withColumn("follower_count", F.when(F.col("follower_count").endswith("k"), expr("substring(follower_count, 1, length(follower_count)-1)") * 1000)
                    .otherwise(F.col("follower_count")))
        .withColumn("follower_count", F.when(F.col("follower_count").endswith("M"), expr("substring(follower_count, 1, length(follower_count)-1)") * 1000000)
                    .otherwise(F.col("follower_count")))
        
        # Clean the data in the save_location column to include only the save location path
        .withColumn("save_location", F.split("save_location", "Local save in"))
        .withColumn("save_location", F.col("save_location")[F.size("save_location") -1])

        .withColumnRenamed("index", "ind")
)

In [0]:
df_pin = (df_pin_cleaned
          .withColumn("follower_count", df_pin_cleaned["follower_count"].cast(IntegerType()))
          .withColumn("index", df_pin_cleaned["ind"].cast(IntegerType()))          
          .select("ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category")          
)          

In [0]:
df_geo = (
    df_geo_pre_cleaning.drop_duplicates([column_name for column_name, data_type in df_geo_pre_cleaning.dtypes])
    .withColumn("coordinates", F.array("latitude", "longitude"))
    .withColumn("timestamp", F.to_timestamp("timestamp"))
    .select("ind", "country", "coordinates", "timestamp")
)

In [0]:
df_user = (
     df_user_pre_cleaning.drop_duplicates([column_name for column_name, data_type in df_user_pre_cleaning.dtypes])
     .withColumn("user_name", F.concat(F.col("first_name"), F.lit(" "), F.col("last_name")))
     .withColumn("date_joined", F.to_timestamp("date_joined"))
     .select("ind", "user_name", "age", "date_joined")    
)

In [0]:
w = Window.partitionBy('country')

(
    df_pin.join(df_geo,
                on="ind",
                how="inner"
                )
    # Group by and add column category_count
    .groupBy("country", "category")
    .agg(F.count("category").alias("category_count"))

    # Get max category per country
    .withColumn('max_count', F.max('category_count').over(w))
    .where(F.col('category_count') == F.col('max_count'))
    .drop('max_count')
    .show(truncate=100)
)


+--------------------------------------------+--------------+--------------+
 country| category|category_count|
+--------------------------------------------+--------------+--------------+
 Afghanistan| education| 5|
 Albania| art| 4|
 Algeria| quotes| 8|
 American Samoa| tattoos| 3|
 Andorra| tattoos| 4|
 Angola|diy-and-crafts| 2|
 Anguilla|diy-and-crafts| 2|
Antarctica (the territory South of 60 deg S)| tattoos| 2|
 Antigua and Barbuda| art| 3|
 Argentina| tattoos| 3|
 Armenia|diy-and-crafts| 3|
 Aruba| mens-fashion| 5|
 Australia| mens-fashion| 2|
 Austria| travel| 2|
 Austria| home-decor| 2|
 Azerbaijan| finance| 1|
 Azerbaijan|event-planning| 1|
 Bahamas| art| 1|
 Bahamas| vehicles| 1|
 Bahamas| travel| 1|
+--------------------------------------------+--------------+--------------+
only showing top 20 rows

In [0]:
w = Window.partitionBy("post_year")

(
    df_pin.join(df_geo,
                on="ind",
                how="inner"
    )
    .withColumn("post_year", F.year("timestamp"))
    .where(F.col("post_year").between("2018", "2022"))

    # Group by and add column category_count
    .groupBy("post_year", "category")
    .agg(F.count("category").alias("category_count"))

    # Get max category per year
    .withColumn("max_count", F.max("category_count").over(w))
    .where(F.col("category_count") == F.col("max_count"))
    .drop("max_count")
    .show()
)

+---------+--------------+--------------+
post_year| category|category_count|
+---------+--------------+--------------+
 2018| art| 10|
 2018| education| 10|
 2019|diy-and-crafts| 10|
 2020| finance| 10|
 2021| quotes| 9|
 2022| christmas| 10|
+---------+--------------+--------------+

In [0]:
w = Window.partitionBy("country")

df_user_with_most_followers_by_country = (
    df_pin.join(df_geo,
                on="ind",
                how="inner"
                )
            # For each country return the user with the most followers
            .drop_duplicates(["country", "poster_name", "follower_count"])
            .select("country", "poster_name", "follower_count")
            .withColumn("max_follower_count", F.max("follower_count").over(w))
            .where(F.col("follower_count") == F.col("max_follower_count"))
            .drop("max_follower_count")
)

df_user_with_most_followers_by_country.show(truncate=100)

+--------------------------------------------+-----------------------------------------------------------------+--------------+
 country| poster_name|follower_count|
+--------------------------------------------+-----------------------------------------------------------------+--------------+
 Afghanistan| 9GAG| 3000000|
 Albania| The Minds Journal| 5000000|
 Algeria| YourTango| 942000|
 American Samoa| Mamas Uncut| 8000000|
 Andorra| Teachers Pay Teachers| 1000000|
 Angola| Tastemade| 8000000|
 Anguilla| Kristen | Lifestyle, Mom Tips & Teacher Stuff Blog| 92000|
Antarctica (the territory South of 60 deg S)| Refinery29| 1000000|
 Antigua and Barbuda| Country Living Magazine| 1000000|
 Argentina| Next Luxury| 800000|
 Armenia| Michelle {CraftyMorning.com}| 892000|
 Aruba| GQ Magazine| 874000|
 Australia| Cultura Colectiva| 1000000|
 Austria| The World Pursuit Travel Website| 89000|
 Azerbaijan| Style Me Pretty| 6000000|
 Bahamas| The Kitchen Table Classroom| 221000|
 Bahrain|R.J. Weiss at The Ways To Wealth | Personal Finance | Debt Payoff| 46000|
 Bangladesh| Better Homes and Gardens| 4000000|
 Barbados| The Creativity Exchange| 410000|
 Belarus| Ricky Lee| 709|
+--------------------------------------------+-----------------------------------------------------------------+--------------+
only showing top 20 rows

In [0]:

(
    df_user_with_most_followers_by_country
    .select("country", "follower_count")
    .sort("follower_count", "country", ascending=[False, True])
    .limit(1)
    .show()
)



+--------------+--------------+
 country|follower_count|
+--------------+--------------+
American Samoa| 8000000|
+--------------+--------------+

In [0]:

w = Window.partitionBy("age_group")

(
    df_pin.join(df_user,
                on="ind",
                how="inner")
    .withColumn('age_group', 
                when( (F.col("age") >= 18) & (F.col("age") < 25), "18-24")
                .when( (F.col("age") >= 25) & (F.col("age") < 36), "25-35")
                .when( (F.col("age") >= 36) & (F.col("age") < 51), "36-50")                
               .otherwise("+50")
               )
    .groupby("age_group", "category")
    .agg(F.count("category").alias("category_count"))
    .withColumn("max_category_count", F.max("category_count").over(w))
    .where(F.col("category_count") == F.col("max_category_count"))
    .drop("max_category_count")
    .show()
)

+---------+------------+--------------+
age_group| category|category_count|
+---------+------------+--------------+
 +50|mens-fashion| 5|
 +50| beauty| 5|
 18-24| tattoos| 20|
 25-35| finance| 12|
 25-35| travel| 12|
 36-50| quotes| 9|
 36-50| home-decor| 9|
+---------+------------+--------------+

In [0]:
magic_percentile = F.expr('percentile_approx(follower_count, 0.5)')

(
    df_pin.join(df_user,
                on="ind",
                how="inner")
    .withColumn('age_group', 
                when( (F.col("age") >= 18) & (F.col("age") < 25), "18-24")
                .when( (F.col("age") >= 25) & (F.col("age") < 36), "25-35")
                .when( (F.col("age") >= 36) & (F.col("age") < 51), "36-50")                
               .otherwise("+50")
               )
    .groupBy('age_group').agg(magic_percentile.alias('median_follower_count'))
    .show()
)

+---------+---------------------+
age_group|median_follower_count|
+---------+---------------------+
 36-50| 6000|
 +50| 3000|
 18-24| 104000|
 25-35| 27000|
+---------+---------------------+

In [0]:
(
    df_user.join(df_geo,
                 on="ind",
                 how="inner")
    .withColumn("joined_year", F.year("date_joined"))
    .where( ((F.col("joined_year") > 2014) & (F.col("joined_year") < 2021)) )
    .withColumn("post_year", F.year("timestamp"))
    .groupby("post_year").agg(F.count(F.col("post_year")).alias("number_users_joined"))
    .sort("post_year", ascending=True)
    .show()
)

+---------+-------------------+
post_year|number_users_joined|
+---------+-------------------+
 2017| 15|
 2018| 78|
 2019| 75|
 2020| 81|
 2021| 67|
 2022| 63|
+---------+-------------------+

In [0]:
magic_percentile = F.expr('percentile_approx(follower_count, 0.5)')

(
    df_user.join(df_pin,
                 on="ind",
                 how="inner")
    .join(df_geo,
          on="ind",
          how="inner")
    .withColumn("joined_year", F.year("date_joined"))
    .where( ((F.col("joined_year") > 2014) & (F.col("joined_year") < 2021)) )
    .withColumn("post_year", F.year("timestamp"))
    .groupBy('post_year').agg(magic_percentile.alias('median_follower_count'))
    .show()
)

+---------+---------------------+
post_year|median_follower_count|
+---------+---------------------+
 2018| 25000|
 2022| 37000|
 2019| 25000|
 2020| 25000|
 2017| 72000|
 2021| 19000|
+---------+---------------------+

In [0]:
magic_percentile = F.expr('percentile_approx(follower_count, 0.5)')

(
    df_user.join(df_pin,
                 on="ind",
                 how="inner")
    .join(df_geo,
          on="ind",
          how="inner")
    .withColumn("joined_year", F.year("date_joined"))
    .where( ((F.col("joined_year") > 2014) & (F.col("joined_year") < 2021)) )
    .withColumn("post_year", F.year("timestamp"))
    .withColumn('age_group', 
                when( (F.col("age") >= 18) & (F.col("age") < 25), "18-24")
                .when( (F.col("age") >= 25) & (F.col("age") < 36), "25-35")
                .when( (F.col("age") >= 36) & (F.col("age") < 51), "36-50")                
               .otherwise("+50")
               )    
    .groupBy('post_year', 'age_group').agg(magic_percentile.alias('median_follower_count'))
    .sort(["post_year", "age_group"], ascending=[True, True])
    .show()
)

+---------+---------+---------------------+
post_year|age_group|median_follower_count|
+---------+---------+---------------------+
 2017| +50| 14000|
 2017| 18-24| 613000|
 2017| 25-35| 31000|
 2017| 36-50| 72000|
 2018| +50| 1000|
 2018| 18-24| 61000|
 2018| 25-35| 22000|
 2018| 36-50| 709|
 2019| +50| 542|
 2019| 18-24| 51000|
 2019| 25-35| 10000|
 2019| 36-50| 6000|
 2020| +50| 53|
 2020| 18-24| 60000|
 2020| 25-35| 27000|
 2020| 36-50| 7000|
 2021| +50| 1000|
 2021| 18-24| 190000|
 2021| 25-35| 26000|
 2021| 36-50| 5000|
+---------+---------+---------------------+
only showing top 20 rows

In [0]:
# Get notebook path
dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()

# Get Databricks cluster id
spark.conf.get("spark.databricks.clusterUsageTags.clusterId")

Out[228]: '1108-162752-8okw8dgg'